## Фреймворк PyTorch для разработки искусственных нейронных сетей.

### Урок 2. Feed-forward neural network.

**1. Сделаем необходимые импорты**

In [3]:
import numpy as np

import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn

from tqdm import tqdm

**2. Загрузим датасет CIFAR-100, сразу же создадим dataloader для него. Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10.**

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

BATCH_SIZE = 4

train_set = torchvision.datasets.CIFAR100(root='./temp_data', train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR100(root='./temp_data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./temp_data/cifar-100-python.tar.gz to ./temp_data
Files already downloaded and verified


In [5]:
def get_img_vector_size(args)->int:
    dataiter = iter(train_loader)
    images, labels = dataiter.next()
    size = images[0].size()
    return size[0] * size[1] * size[2]

**3.  Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток). Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x). Ваша задача добавить его в архитектуру сети как функцию активации.**

In [6]:
class CustomLeakyRelu(nn.Module):
    def __init__(self, border:float, alpha:float) -> None:
        super().__init__()
        self.border = border
        self.alpha = alpha
        
    def forward(self, input):
        x = F.leaky_relu(input)
        return torch.where(x>self.border, x*self.alpha, x)

class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        img_vector_size = 3072
        self.fc1 = nn.Linear(img_vector_size,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,100)
        self.cust_relu = CustomLeakyRelu(0.1, 0.5)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)        
        x = self.cust_relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        x = self.cust_relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
        
net = Net()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

**4. Запустить обучение (по аналогии с тем, что делали на паре).**

In [7]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0 
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        optimizer.zero_grad() # Обнуление градиента

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() # Вывод статистики о процессе обучения
        if i % 300 == 0:    # Вывод каждых 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('End of training.')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.002
[1,   301] loss: 0.685
[1,   601] loss: 0.651
[1,   901] loss: 0.636
[1,  1201] loss: 0.629
[1,  1501] loss: 0.619
[1,  1801] loss: 0.605
[1,  2101] loss: 0.602
[1,  2401] loss: 0.606
[1,  2701] loss: 0.604
[1,  3001] loss: 0.594
[1,  3301] loss: 0.598
[1,  3601] loss: 0.602
[1,  3901] loss: 0.582
[1,  4201] loss: 0.582
[1,  4501] loss: 0.579
[1,  4801] loss: 0.583
[1,  5101] loss: 0.588
[1,  5401] loss: 0.591
[1,  5701] loss: 0.579
[1,  6001] loss: 0.574
[1,  6301] loss: 0.578
[1,  6601] loss: 0.577
[1,  6901] loss: 0.590
[1,  7201] loss: 0.570
[1,  7501] loss: 0.570
[1,  7801] loss: 0.572
[1,  8101] loss: 0.567
[1,  8401] loss: 0.575
[1,  8701] loss: 0.564
[1,  9001] loss: 0.567
[1,  9301] loss: 0.563
[1,  9601] loss: 0.561
[1,  9901] loss: 0.572
[1, 10201] loss: 0.571
[1, 10501] loss: 0.567
[1, 10801] loss: 0.558
[1, 11101] loss: 0.569
[1, 11401] loss: 0.554
[1, 11701] loss: 0.563
[1, 12001] loss: 0.550
[1, 12301] loss: 0.572


 10%|█         | 1/10 [01:44<15:37, 104.13s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.548
[2,   601] loss: 0.555
[2,   901] loss: 0.550
[2,  1201] loss: 0.557
[2,  1501] loss: 0.562
[2,  1801] loss: 0.557
[2,  2101] loss: 0.541
[2,  2401] loss: 0.541
[2,  2701] loss: 0.558
[2,  3001] loss: 0.544
[2,  3301] loss: 0.554
[2,  3601] loss: 0.556
[2,  3901] loss: 0.543
[2,  4201] loss: 0.551
[2,  4501] loss: 0.556
[2,  4801] loss: 0.547
[2,  5101] loss: 0.553
[2,  5401] loss: 0.543
[2,  5701] loss: 0.552
[2,  6001] loss: 0.553
[2,  6301] loss: 0.549
[2,  6601] loss: 0.544
[2,  6901] loss: 0.556
[2,  7201] loss: 0.551
[2,  7501] loss: 0.556
[2,  7801] loss: 0.535
[2,  8101] loss: 0.550
[2,  8401] loss: 0.542
[2,  8701] loss: 0.553
[2,  9001] loss: 0.562
[2,  9301] loss: 0.559
[2,  9601] loss: 0.558
[2,  9901] loss: 0.551
[2, 10201] loss: 0.551
[2, 10501] loss: 0.559
[2, 10801] loss: 0.560
[2, 11101] loss: 0.555
[2, 11401] loss: 0.546
[2, 11701] loss: 0.543
[2, 12001] loss: 0.540
[2, 12301] loss: 0.556


 20%|██        | 2/10 [03:26<13:42, 102.87s/it]

...

[9,     1] loss: 0.002
[9,   301] loss: 0.515
[9,   601] loss: 0.520
[9,   901] loss: 0.526
[9,  1201] loss: 0.529
[9,  1501] loss: 0.515
[9,  1801] loss: 0.519
[9,  2101] loss: 0.543
[9,  2401] loss: 0.522
[9,  2701] loss: 0.529
[9,  3001] loss: 0.521
[9,  3301] loss: 0.522
[9,  3601] loss: 0.525
[9,  3901] loss: 0.525
[9,  4201] loss: 0.508
[9,  4501] loss: 0.531
[9,  4801] loss: 0.529
[9,  5101] loss: 0.514
[9,  5401] loss: 0.520
[9,  5701] loss: 0.520
[9,  6001] loss: 0.526
[9,  6301] loss: 0.525
[9,  6601] loss: 0.530
[9,  6901] loss: 0.519
[9,  7201] loss: 0.542
[9,  7501] loss: 0.527
[9,  7801] loss: 0.532
[9,  8101] loss: 0.543
[9,  8401] loss: 0.534
[9,  8701] loss: 0.525
[9,  9001] loss: 0.524
[9,  9301] loss: 0.528
[9,  9601] loss: 0.524
[9,  9901] loss: 0.525
[9, 10201] loss: 0.535
[9, 10501] loss: 0.531
[9, 10801] loss: 0.536
[9, 11101] loss: 0.531
[9, 11401] loss: 0.532
[9, 11701] loss: 0.531
[9, 12001] loss: 0.543
[9, 12301] loss: 0.530


 90%|█████████ | 9/10 [16:00<01:48, 108.61s/it]

[10,     1] loss: 0.002
[10,   301] loss: 0.515
[10,   601] loss: 0.520
[10,   901] loss: 0.515
[10,  1201] loss: 0.523
[10,  1501] loss: 0.513
[10,  1801] loss: 0.508
[10,  2101] loss: 0.518
[10,  2401] loss: 0.517
[10,  2701] loss: 0.511
[10,  3001] loss: 0.524
[10,  3301] loss: 0.530
[10,  3601] loss: 0.522
[10,  3901] loss: 0.523
[10,  4201] loss: 0.524
[10,  4501] loss: 0.528
[10,  4801] loss: 0.532
[10,  5101] loss: 0.526
[10,  5401] loss: 0.526
[10,  5701] loss: 0.534
[10,  6001] loss: 0.523
[10,  6301] loss: 0.515
[10,  6601] loss: 0.521
[10,  6901] loss: 0.531
[10,  7201] loss: 0.519
[10,  7501] loss: 0.527
[10,  7801] loss: 0.530
[10,  8101] loss: 0.533
[10,  8401] loss: 0.538
[10,  8701] loss: 0.535
[10,  9001] loss: 0.530
[10,  9301] loss: 0.530
[10,  9601] loss: 0.522
[10,  9901] loss: 0.528
[10, 10201] loss: 0.523
[10, 10501] loss: 0.545
[10, 10801] loss: 0.525
[10, 11101] loss: 0.526
[10, 11401] loss: 0.517
[10, 11701] loss: 0.535
[10, 12001] loss: 0.512
[10, 12301] loss

100%|██████████| 10/10 [17:46<00:00, 106.68s/it]

End of training.
